<a href="https://colab.research.google.com/github/junjunjanna/assignment/blob/main/usyd_phys_5020_img_reg_seg_assignment_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PHYS 5020 - Image Registration and Segmentation

## Assignment Worksheet

This jupyter notebook contains useful pieces of documented python code.

Please feel free to use, adapt, and extend this code in your assignment.

You are encouraged to add to this notebook!

### Links
To download and check out platipy: https://github.com/pyplati/platipy

You should check out the examples: https://github.com/pyplati/platipy/tree/master/examples

Documentation available here: https://pyplati.github.io/platipy/


### Authors:
Dr Rob Finnegan

Dr John Kipritidis

### Student information:

#### Name:
*name here*

#### Student number:
*number here*

#### Date:
date here

In [ ]:
# set-up
# IMPORTANT - IF YOU ARE USING GOOGLE COLAB MAKE SURE TO RESTART RUNTIME!

try:
    import platipy
except ImportError:
    !pip install platipy

In [ ]:
# import some basic modules

from platipy.imaging import ImageVisualiser

import SimpleITK as sitk
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# download the assignment data

!gdown 1R6VbCJ7xUxF5PvoGLbBKSvIC_ZGKcyhe --output data.zip

In [ ]:
# unzip the data
# you can change the output location if you want

!unzip ./data.zip > /dev/null

***
# Task 1 
## Automatic bone segmentation on CT scans 

Segmentation of bones can be used to assess patient set-up, which is relevant for radiotherapy treatment. 

This task can be accomplished with simple methods because the x-ray attenuation of bones is much greater than that of soft tissues, and so the image intensity values are much higher for bone than other soft tissues.



As a medical physics registrar, you have been provided with two CT scans from the same patient, one of the head and neck region (HN) and one of the body (from the neck to mid-thigh).

### Question 1

Generate a histogram of image intensity values using the HN scan.

Include on your axes the physical units used to measure CT image values.

In [ ]:
# read in the HN CT scan

img_ct_hn = sitk.ReadImage("./data/task_1/HN_CT.nii.gz")

In [ ]:
# visualise the image
vis = ImageVisualiser(
    image = img_ct_hn,
    cut = (95,256,256),         # the (axial, coronal, sagittal) slice location
    axis = 'ortho',             # this can be "z (axial)", "y (corornal)", "x (sagittal)", "ortho"
    window = (-250, 500),       # specify as (min, range)
    figure_size_in = 6,         # the size of the image
    colormap = plt.cm.Greys_r,  # color map, see: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    projection = False,         # type of intensity projection: "max" (MIP), "mean", "median", etc.
)
    
# create the figure
fig = vis.show()

In [ ]:
# read in the contours
# these are also just an image!

contour_external = sitk.ReadImage("./data/task_1/HN_RTSTRUCT_EXTERNAL.nii.gz")

In [ ]:
# visualise contours

vis = ImageVisualiser(
    image = img_ct_hn,
    cut = (95,256,256),
    axis = 'ortho',
    window = (-250, 500),
    figure_size_in = 6,
    colormap = plt.cm.Greys_r,
    projection = False,
)
    
# add contours
vis.add_contour(
    contour = {"EXTERNAL":contour_external},
    color= {"EXTERNAL":"red"},
    linewidth=2,
    show_legend=True
)
    
    
fig = vis.show()

In [ ]:
# extract the image values (as a numpy array)
vals_ct_hn = sitk.GetArrayFromImage(img_ct_hn)

# extract the contour (mask)
vals_contour_external = sitk.GetArrayFromImage(contour_external)

# just analyse voxels inside the patient
vals_inside_patient = vals_ct_hn[np.where(vals_contour_external)]

# plot a histogram
fig, ax = plt.subplots(1,1)

ax.hist(
    vals_inside_patient.flatten(),
    bins = np.linspace(-1000,1500,500), # make 500 bins between -1000 and 1500
    color = "red", # change to something else if you like!
)

# plotting options

ax.set_yscale("log") # choose from "log", "linear", etc.
ax.set_xlabel("INSERT CORRECT X LABEL")
ax.set_ylabel("INSERT CORRECT Y LABEL")
ax.set_title("INSERT CORRECT TITLE")

# more options
ax.grid(); ax.set_axisbelow(True)

fig.tight_layout()
fig.show()

#### Describe the types of materials that correspond different peaks in this graph.

put your answer here

### Question 2
By varying the intensity windowing and slice position, generate a 2D sagittal image from the HN CT scan in which the metal plate in the neck can be clearly observed.

In [ ]:
# modify the value of `cut` and `window` as appropriate.
# i.e. replace ? with valid numbers!

vis = ImageVisualiser(
    image = img_ct_hn,
    cut = ?,
    axis = 'x',
    window = (?,?),
    figure_size_in = 6,
    colormap = plt.cm.Greys_r,
    projection = False,
)
    
fig = vis.show()

### Question 3
Using an appropriate threshold, create a segmentation of all the voxels in the HN CT scan that have an attenuation similar to bone. 

Visualise this segmentation, and describe the results.

In [ ]:
# thresholding an image to create a mask
# replace ? with correct values to match the CT number of bone
bone_mask_threshold = sitk.BinaryThreshold(img_ct_hn, lowerThreshold=?, upperThreshold=?) 

In [ ]:
# we can visualise the segmentation as a scalar overlay
# this can be more helpful than contours!

vis = ImageVisualiser(
    image = img_ct_hn,
    cut=(33,256,256),
    figure_size_in=6
)
    
vis.add_scalar_overlay(
    bone_mask_threshold,
    name = "TEST SEGMENTATION",
    alpha = 0.75,
    min_value = 0,
    max_value = 1,
    colormap = plt.cm.Greens
)
    
    
fig = vis.show()

#### Describe the results.

put you answer here

### Question 4
Using at least one post-processing algorithm, produce an improved bone segmentation, and describe your choices and reasoning.

Examples of post-processing include connected component analysis and morphological operations (hole filling, closing, etc.). 

Do not perform any manual editing.


In [ ]:
# connected component analysis

# run the simpleitk function
bone_mask_con_com = sitk.RelabelComponent(sitk.ConnectedComponent(bone_mask_threshold))

# YOU NEED TO VISUALISE THE CONNECTED COMPONENTS TO CHOOSE
vis = ImageVisualiser(
    image = img_ct_hn,
    cut=(33,256,256),
    figure_size_in=6
)

vis.add_scalar_overlay(
    bone_mask_con_com,
    name = "LABEL VALUES",
    alpha = 0.75,
    min_value = 0,
    max_value = 6,
    discrete_levels=7,
    mid_ticks=True,
    colormap = plt.cm.hsv
)

fig = vis.show()

In [ ]:
# now manually choose the correct labels
# uncomment if you need more than one label
bone_mask_select = (bone_mask_con_com==?) # + (bone_mask_con_com==?) + (bone_mask_con_com==?)

# display as a contour or scalar overlay
vis = ImageVisualiser(
    image = img_ct_hn,
    cut=(33,256,256),
    figure_size_in=6
)
    
vis.add_contour(
    contour = {"Bone segmentation":bone_mask_select},
    color= {"Bone segmentation":"cyan"}, # see: https://matplotlib.org/stable/gallery/color/named_colors.html for a full list
)
    
fig = vis.show()

In [ ]:
# morphological operations

# run the simpleitk function
bone_mask_morph = sitk.BinaryMorphologicalClosing(bone_mask_select, (2,2,4)) # the last numbers are the number of voxels to erode in the (sagittal, coronal, axial) directions

# other options
# sitk.BinaryErode : shrinks a segmentation
# sitk.BinaryDilate : the opposite of erode, expands a segmentation
# sitk.BinaryFillhole: fills small holes
# sitk.BinaryMorphologicalClosing: a more complex hole filling algorithm
#                                  specify the max hole size: (sag, cor, ax)

# display as a contour or scalar overlay
vis = ImageVisualiser(
    image = img_ct_hn,
    cut=(33,256,256),
    figure_size_in=6
)
    
vis.add_contour(
    contour = {"INSERT NAME HERE":bone_mask_morph},
    color= {"INSERT NAME HERE":"orange"},
)
    
fig = vis.show()

#### Describe your choice of algorithm/s and reasoning.

put your answer here

### Question 5

Apply your bone segmentation method to the body scan. 

Include visualisations (at least one set of orthogonal image slices or MIP slices) to support your answer.

In [ ]:
# now repeat for the body scan

img_ct_body = sitk.ReadImage("./data/task_1/BODY_CT.nii.gz")

In [ ]:
# put your code here

#### Discuss the performance on this new image.

put you answer here

#### Are any changes required to work on this new image? 

put your answer here

***
# Task 2
## CT-CT image registration and 18FDG-PET analysis.

A patient has received $^{18}$FDG-PET scan as part of their treatment for head and neck cancer, to better visualise metabolically active tumour regions. 

This PET scan also includes a CT that is used for attenuation correction, and both images together are referred to as the PET-CT. 

With advice from their medical team, this patient has decided to receive radiotherapy.

Prior to radiotherapy the patient has recieved a simulation CT scan (i.e. set up in the same position as for treatment).

A radiation oncologist has delineated the primary tumour (gross tumour volume, GTV). 

When the PET-CT scan was acquired, the patient position was slightly different to the simulation CT. 

As a medical physicist, you have been asked to help with the co-registration of the PET-CT to the simulation CT.

In [ ]:
# we need to import more functionality

from platipy.imaging.registration.linear import linear_registration
from platipy.imaging.registration.deformable import fast_symmetric_forces_demons_registration
from platipy.imaging.registration.utils import apply_transform

from platipy.imaging.label.utils import get_com # get the centre of mass

In [ ]:
# read in the data for this task

# the simulation ct and contour
img_ct_sim = sitk.ReadImage("./data/task_2/HN_CT_SIMULATION.nii.gz")
contour_gtv = sitk.ReadImage("./data/task_2/HN_RTSTRUCT_TUMOUR.nii.gz")

img_ct_ac = sitk.ReadImage("./data/task_2/HN_CT_AC.nii.gz")
img_pet = sitk.ReadImage("./data/task_2/HN_PET.nii.gz")

# VERY IMPORTANT
# THE PET IMAGE HAS A DIFFERENT RESOLUTION
# SO WE MUST RESAMPLE IT INTO THE SAME SPACE AS THE CT
img_pet = sitk.Resample(img_pet, img_ct_ac)

In [ ]:
# visualise the planning CT

vis = ImageVisualiser(
    image = img_ct_sim,
    cut = get_com(contour_gtv), # a useful way to specify the slice location
    figure_size_in = 6
)

vis.add_contour(
    contour = {"INSERT NAME":contour_gtv},
    color= {"INSERT NAME":"lime"},
    show_legend=True
)

fig = vis.show()

### Question 1
By manually adjusting the slice location of the PET scan, visualise a 2D image slice from the PET-CT at the GTV location. 

In [ ]:
# put your code here

#### Describe the appearance of the GTV on the PET scan, and discuss potential the cause of this appearance.

put you answer here

### Question 2

Use a linear registration method to align the attenuation correction CT to the simulation CT.

The valid options for `reg_method`:
- `"translation"`
- `"rigid"`
- `"similarity"`
- `"affine"`
- `"scaleversor"` (rigid + anisotropic scale)
- `"scaleskewversor"` (affine + anisotropic scale)


The valid options for `metric`:
- `"mean_squares"`
- `"correlation"`
- `"mattes_mi"` (MI = mutual information)
- `"joint_hist_mi"` (MI = mutual information)

The valid options for `optimiser`:
- `"gradient_descent"` (usually the best option)
- `"gradient_descent_line_search"`
- `"lbfgsb (limited-memory Broyden–Fletcher–Goldfarb–Shanno (bounded).)"`

Note: you shouldn't have to change the optimisation option, but if a registration is failing you can try out the other choices.


In [ ]:
# to get more info:
linear_registration?

In [ ]:
# linear registration

img_ct_ac_linear, tfm_ct_ac_linear = linear_registration(
    fixed_image = img_ct_sim,
    moving_image = img_ct_ac,
    reg_method = ?, # replace this with a valid option
    metric = ?, # replace this with a valid option
    # optimiser = ?, # replace this with a valid option
    shrink_factors = [8, 4, 2], # try out different options
    smooth_sigmas = [4, 2, 0], # try out different options
    sampling_rate = 0.5, # between 0 and 1. increase = better accuracy, but slower
    default_value = -1000, # the "background", for CT this is -1000
    verbose = False, # change to True to print the progress
)

In [ ]:
# visualise the planning CT
vis = ImageVisualiser(
    image = img_ct_sim,
    figure_size_in = 6
)

# add the AC CT as a comparison
vis.add_comparison_overlay(img_ct_ac_linear)

fig = vis.show()

#### Discuss why you chose the registration parameters you used, in particular the type of transform and similarity metric. 

put your answer here

#### Describe the co-registration accuracy, using visualisations as appropriate.

put your answer here (create visualisations above)

### Question 3
Use a deformable registration method to refine these results.

In [ ]:
# read the doc string for more info!
fast_symmetric_forces_demons_registration?

In [ ]:
# deformable registration

img_ct_ac_dir, tfm_ct_ac_dir, dvf_ct_ac = fast_symmetric_forces_demons_registration(
    fixed_image = img_ct_sim,
    moving_image = img_ct_ac_linear,
    resolution_staging=[8, 4, 2], # this is the resolution in mm
    iteration_staging=[10, 10, 10], # more iterations = better accuracy but slover
    isotropic_resample=True, # usually best to set this to True
    default_value=-1000,
    verbose=False,
)

In [ ]:
# visualise the DIR results
vis = ImageVisualiser(
    image = img_ct_sim,
    figure_size_in = 6
)

vis.add_comparison_overlay(img_ct_ac_dir)

fig = vis.show()

#### Describe the co-registration accuracy following this second registration stage, using visualisations as appropriate.

put your answer here

### Question 4
Propagate the transform to the PET image, and visualise the simulation CT with the PET overlaid and the GTV shown as a contour.
[check TG-132 language]

In [ ]:
# we will use the `apply_transform` function, get some information about it:
apply_transform?

In [ ]:
# combine the chain of transforms
tfm_ct_ac_total = sitk.CompositeTransform((tfm_ct_ac_linear, tfm_ct_ac_dir))

# apply to the pet image
img_pet_dir = apply_transform(img_pet, reference_image=img_ct_sim, transform=tfm_ct_ac_total)

In [ ]:
# visualise!

#### Describe the alignment between the tumour region as visible on the PET scan with the delineated GTV.

put your answer here

### Question 5

Compute the total activity (in Bq) of the tumour volume. Compare this to the total activity as provided by the ARPANSA Diagnostic Reference Level (DRL) for a brain PET-CT, found [here](https://www.arpansa.gov.au/research-and-expertise/surveys/national-diagnostic-reference-level-service/current-australian-drls/nm).

For this question, consider how the CT image values were extracted in Task 1, Question 1 to compute the histogram. Some steps:
 - Using a similar approach we can extract the activity concentration at each voxel
 - How could we compute the *activity* at each voxel?
 - Hint: compute the volume of the voxel: `voxel_vol = np.product(img_pet_dir.GetSpacing())`. 

In [ ]:
# put your code here

#### written response

put your answer here

***
# Task 3
## DIR-based functional lung assessment

Deformable image registration can be used to assess lung function, as an alternative to SPECT- or PET-based ventilation and perfusion imaging (e.g. using $^{99\text{m}}$Tc or $^{68}$Ga).

The basis of these DIR methods is measuring changes in images of the lungs in 4D CT scans.

A radiotherapy patient has received a 4D CT scan, gated into 10 phases of the respiratory cycle. 

From this scan, two CT scans corresponding to the peak inhale (breath in) and peak exhale (breath out) are generated.

As a medical physicist registrar, you have been asked to help provide quantitative information. 

This will help the team in implementing a new technique to minimise the radiation dose to well-functioning lung regions.


In [ ]:
# read in the data for this task

img_4dct_inhale = sitk.ReadImage("./data/task_3/4DLUNG_CT_INHALE.nii.gz")
img_4dct_exhale = sitk.ReadImage("./data/task_3/4DLUNG_CT_EXHALE.nii.gz")

contours = {"EXHALE":{}, "INHALE":{}}
contours["EXHALE"]["LEFT_LUNG"] = sitk.ReadImage("./data/task_3/4DLUNG_RTSTRUCT_LEFT_LUNG_EXHALE.nii.gz")
contours["EXHALE"]["RIGHT_LUNG"] = sitk.ReadImage("./data/task_3/4DLUNG_RTSTRUCT_RIGHT_LUNG_EXHALE.nii.gz")
contours["INHALE"]["LEFT_LUNG"] = sitk.ReadImage("./data/task_3/4DLUNG_RTSTRUCT_LEFT_LUNG_INHALE.nii.gz")
contours["INHALE"]["RIGHT_LUNG"] = sitk.ReadImage("./data/task_3/4DLUNG_RTSTRUCT_RIGHT_LUNG_INHALE.nii.gz")

In [ ]:
# visualise

vis = ImageVisualiser(img_4dct_exhale, cut=get_com(contours["EXHALE"]["LEFT_LUNG"]), figure_size_in=6)
vis.add_contour(contours["EXHALE"])
fig = vis.show()

### Question 1
Use deformable image registration to warp the peak exhale CT image to the peak inhale CT image.

In [ ]:
# register using the fast symmetric forces demons algorithm!

img_4dct_dir, tfm_4dct_dir, dvf_4dct = fast_symmetric_forces_demons_registration(
    fixed_image = img_4dct_inhale,
    moving_image = img_4dct_exhale,
    resolution_staging=[8, 4, 2],
    iteration_staging=[10, 10, 10],
    isotropic_resample=True,
    default_value=-1000,
    verbose=False,
)

Compare this warped exhale image to the peak inhale image

In [ ]:
# compare the registration
# BEFORE REG
vis = ImageVisualiser(img_4dct_inhale, axis="y", figure_size_in=4)
vis.add_comparison_overlay(img_4dct_exhale)
fig = vis.show()

# AFTER REF
vis = ImageVisualiser(img_4dct_inhale, axis="y", figure_size_in=4)
vis.add_comparison_overlay(img_4dct_dir)
fig = vis.show()

#### Describe the accuracy of the registration process, using visualisations as required.

put your answer here

### Question 2
Compute the magnitude of the deformation field, mask it to the lung volume.

In [ ]:
# calculate the magnitude
img_vec_mag = sitk.VectorMagnitude(dvf_4dct)

# mask the the combined lung volume
img_vec_mag_masked = sitk.Mask(img_vec_mag, contours["INHALE"]["RIGHT_LUNG"] | contours["INHALE"]["LEFT_LUNG"])

Visualise this as a scalar overlay on the inhale CT scan using a perceptually uniform colormap.

To find an appropriate colormap see: https://matplotlib.org/stable/tutorials/colors/colormaps.html

In [ ]:
# visualise
# replace ? with a valid PERCENTUALLY UNIFORM colormap

vis = ImageVisualiser(img_4dct_inhale, cut=get_com(contours["INHALE"]["LEFT_LUNG"]), figure_size_in=6)

vis.add_scalar_overlay(
    img_vec_mag_masked,
    colormap = ?,
    discrete_levels = 10,
    max_value = 20,
    name = "INSERT NAME"
)

fig = vis.show()

#### Where are the largest deformations? Is this what you expected?

put your answer here

### Question 3
Compute the Jacobian determinant of the deformation field, mask it to the lung volume.

In [ ]:
# calculate the jacobian determinant
img_jac_det = sitk.DisplacementFieldJacobianDeterminant(sitk.InvertDisplacementField(dvf_4dct))

In [ ]:
img_jac_det_mask = sitk.Mask(img_jac_det, contours["INHALE"]["RIGHT_LUNG"] | contours["INHALE"]["LEFT_LUNG"])

Visualise this as a scalar overlay on the inhale CT scan (with lung contours) using a diverging colormap normalised with 1 at the center.

In [ ]:
# create a normalised colormap

from matplotlib import colors
cmap_norm = colors.TwoSlopeNorm(vcenter=1)

In [ ]:
# visualise as a scalar overlay
# replace ? with a valid DIVERGING colormap

vis = ImageVisualiser(img_4dct_inhale, cut=get_com(contours["INHALE"]["LEFT_LUNG"]), figure_size_in=6)

vis.add_scalar_overlay(
    img_jac_det_mask,
    colormap = ?,
    discrete_levels = 10,
    min_value = 0,
    max_value = 3,
    name = "INSERT NAME",
    norm = cmap_norm
)

fig = vis.show()

### Question 4

#### With reference to the above results, briefly describe how the lungs change during respiration. 

put your answer here

#### Based on the analysis of the deformable image registration, are there any regions associated with non-physical deformation? 
Hint: which of the above metrics (vector magnitude, Jacobian determinant) can we used to identify non-physical deformations?

#### What proportion of the total lung volume is affected by this non-physical deformation?
Hint: can you extract all of the appropriate metric values in the lung regions?

In [ ]:
# put your code here

### answer

put your answer here

***
# Task 4
## MRI-CT fusion for brain imaging

It can be beneficial to delineate radiotherapy planning volumes using CT imaging together with another modality, such as MRI, for improved soft-tissue contrast and functional imaging.

A radiation oncologist is treating a patient for brain cancer. 

In addition to the simulation CT acquired to facilitate treatment planning, there were two MRI sequences acquired: T2w FSE (fast spin echo) and T2w FLAIR (fluid-attenuated attenuated inversion recovery). 

As a medical physicist, you have been asked to provide assistance in the fusion (co-registration) of the brain MRI and CT scan. 

This will improve the visualisation of the brain tumour, which could lead to more confidence in the delineation of radiotherapy planning volumes.

In [ ]:
# read in the data for this question

img_brain_ct = sitk.ReadImage("./data/task_4/BRAIN_CT.nii.gz")
img_brain_mri_fse = sitk.ReadImage("./data/task_4/BRAIN_MR_AX_T2_FSE.nii.gz")
img_brain_mri_flair = sitk.ReadImage("./data/task_4/BRAIN_MR_AX_T2_FLAIR.nii.gz")

### Question 1
Which image (CT, T2w FSE, or T2w FLAIR) should be chosen as the reference image (i.e. the target image), and which should be selected as the moving image (i.e. the image that is transformed), and why? 

For choosing which MRI sequence might be preferred, you should investigate acquisition/image parameters.

In [ ]:
# to get information about an image, use
print("Image resoltuion:", img_brain_ct.GetSpacing())
print("Image size:      ", img_brain_ct.GetSize())
print("Image pixel type:", img_brain_ct.GetPixelIDTypeAsString())

#### answer

put you answer here

### Question 2
Use an appropriate co-registration algorithm to align the two modalities. For this, you may use either MRI sequence. 

In [ ]:
# perform linear registration
# replace ? with valid options!
img_brain_registered, tfm_brain = linear_registration(
    fixed_image = ?,
    moving_image = ?,
    reg_method=?,
    metric=?,
    optimiser="gradient_descent",
    shrink_factors=[8,4,2],
    smooth_sigmas=[4,2,0],
    sampling_rate=0.9,
    default_value=?,
    verbose=False,
)

In [ ]:
# visualise as a comparison overlay
# replace ? with your chosen reference (fixed) image
vis = ImageVisualiser(?, figure_size_in=6)
vis.add_comparison_overlay(img_brain_registered)
fig = vis.show()

In [ ]:
# visualise as a scalar overlay
vis = ImageVisualiser(?, figure_size_in=6)

vis.add_scalar_overlay(
    ?,
    colormap=plt.cm.CMRmap,
    name = "INSERT NAME",
    alpha=0.5
)

fig = vis.show()

#### Discuss the registration parameters you chose, in particular, the type of transform and metric.

put your answer here

#### By using appropriate visualisations, describe the accuracy of the co-registration.

put your answer here

***
# Task 5
## Atlas-based automatic segmentation of radiotherapy volumes 

Atlas-based techniques are frequently used in radiotherapy to assist treatment planning.

A cancer therapy centre is currently investigating the how automatic segmentation could be used for patient being treated for head and neck cancers.

These patients routinely receive T2w MRI scans. 

Currently, these is a focus on the parotid glands, which are the largest salivary glands and are important during radiotherapy planning as doses exceeding established constraints can increase the risks of severe toxicities.

As a medical physicist, you have been asked to provide assistance in validating an atlas-based automatic segmentation tool. 

You have collected data (imaging and contours) for three patients, and will conduct an investigation.

### Question 1
Visualise all three patient images with the contours clearly visible.

In [ ]:
# read in data for one patient

img_001 = sitk.ReadImage("./data/task_5/RTMAC_LIVE_001/IMAGES/RTMAC_LIVE_001_MRI_T2W.nii.gz")

contours_001 = {
    "PAROTID_L":sitk.ReadImage("./data/task_5/RTMAC_LIVE_001/STRUCTURES/RTMAC_LIVE_001_RTSTRUCT_PAROTID_L.nii.gz"),
    "PAROTID_R":sitk.ReadImage("./data/task_5/RTMAC_LIVE_001/STRUCTURES/RTMAC_LIVE_001_RTSTRUCT_PAROTID_R.nii.gz"),
}

# repeat for the other two patients

In [ ]:
# visualise
vis = ImageVisualiser(img_001, cut=get_com(contours_001["PAROTID_L"]), figure_size_in=6)
vis.add_contour(contours_001)
fig = vis.show()

# repeat for the other two patients

#### Discuss anatomical and appearance difference between the patients.

put answer here

### Question 2
Select one patient as the “target” to be automatically segmented using the other two datasets as atlases. 

Use the multi-atlas segmentation tool in platipy.

In [ ]:
# select the target image
# replace ? with the patient id (1,2 or 3)
img_target = img_00?
contours_target = contours_00?

In [ ]:
# import the relevant functionality
from platipy.imaging.projects.multiatlas.run import run_segmentation, MUTLIATLAS_SETTINGS_DEFAULTS

from platipy.imaging.visualisation.comparison import contour_comparison

In [ ]:
# display all of the available options
display(MUTLIATLAS_SETTINGS_DEFAULTS)

In [ ]:
# make a copy of the default settings
user_settings = MUTLIATLAS_SETTINGS_DEFAULTS

# we MUST change the atlas settings
# CHANGE X AND Y to the IDs of the two patients you are using as the atlases
user_settings['atlas_settings'] = {
    'atlas_id_list': ['RTMAC_LIVE_00X', "RTMAC_LIVE_00Y"],
    'atlas_structure_list': ['PAROTID_L','PAROTID_R'],
    'atlas_path': './data/task_5/',
    'atlas_image_format': '{0}/IMAGES/{0}_MRI_T2W.nii.gz',
    'atlas_label_format': '{0}/STRUCTURES/{0}_RTSTRUCT_{1}.nii.gz',
    'crop_atlas_to_structures': False,
    'crop_atlas_expansion_mm': None,
}

# optionally, we can change some of the default registration parameters
user_settings["linear_registration_settings"] = {
    "reg_method": "rigid",
    "shrink_factors": [16, 8, 4],
    "smooth_sigmas": [0, 0, 0],
    "sampling_rate": 0.75,
    "default_value": 0,
    "number_of_iterations": 50,
    "metric": "mean_squares",
    "optimiser": "gradient_descent_line_search",
    "verbose": False
}

user_settings["deformable_registration_settings"] = {
    "isotropic_resample": True,
    "resolution_staging": [6, 4],
    "iteration_staging": [25, 25],
    "smoothing_sigmas": [0, 0],
    "ncores": 8,
    "default_value": 0,
    "verbose": False
}

In [ ]:
# Run the segmentation
# this usually takes a few minutes
output_contours, output_probability = run_segmentation(img_target, user_settings)

### Discuss your reasoning in selecting the registration parameters you chose.

put answer here

### Question 3
Generate a figure comparing the manual contours to the automatic segmentation along with measures of geometric similarity (i.e. contouring metrics). 

Notes:
 - DSC: Dice Similarity Coefficient.
 - MDA: Mean Distance to Agreement. Sometimes called Mean Surface Distance (MDA).
 - HD: Hausdorff Distance (maximum distance between contours).

In [ ]:
fig = contour_comparison(
    img = img_target,
    contour_dict_a = contours_target,
    contour_dict_b = output_contours,
    contour_label_a='Manual',
    contour_label_b='Automatic',
    title='INSERT TITLE',
    subtitle='INSERT SUBTITLE',
    subsubtitle='INSERT YOUR NAME',
    img_vis_kw=dict(figure_size_in=9),
)

#### Briefly describe the performance of the atlas-based segmentation tool for this patient.

### Question 4

#### With reference to the recent paper:
[Kieselmann et al (2020) Auto-segmentation of the parotid glands on MR images of head and neck cancer patients with deep learning strategies.](https://www.medrxiv.org/content/10.1101/2020.12.19.20248376v1.full-text)
#### Compare the performance of your segmentation to state-of-the-art results. 

put answer here

#### Would you recommend this tool for use? Why/why not?
put answer here

#### What further validation would you suggest?
put answer here

# Generating a PDF to submit

To create a PDF:
1. Download this Jupyter notebook (File > Download > Download .ipynb)
2. Upload to you workspace (click the folder icon on the left, drag and drop the .ipynb file)
3. Run the two cells below

In [ ]:
%%capture

!sudo apt-get install texlive-xetex texlive-fonts-recommended texlive-generic-recommended

In [ ]:
# replace the name with the actual filename!

!jupyter nbconvert --to pdf /content/NAME_OF_YOUR_FILE.ipynb